pandas  
1) pandas의 기본적인 동작의 특징 : 2D 가로 인덱스, 세로 인덱스 -> 인덱스!  
2) 여러 형식의 데이터들을 입출력에 하기에 용이하게 만들어둠  
    => read_~~: csv, excel, etc R, SAS etc
    => web : json, xml + 외부 패키지(json, bs4)  
    => 개별 데이터 수집은 df은 무거운 부분이 있어서 리스트나 dict etc    
3) 여러 종류, 여러 df, 여러 table 에서 -> 내가 필요한 정보들만 모아서 하나의 df로 만드는 과정    
    => 단순히 합치 : concat  
    => 특정 조건에 맞춰서 합치자 : sql join -> pd.merge(~~)  
4) 필요한 데이터만 딱 정리를 한 상태!  
    => 내가 보고 싶은 대로 데이터의 모양에 대한 변경!  
기존은 샘플별로 하나씩 자료를 수집했다고 하면  
이번에는 주로 요소별로 보려고 할 때 => 엑셀 피벗 테이블
    => 각각의 데이터/ 샘플별로 보는 것이 아니라 내가 원하는 기준대로 샘플이 어떤지 보는 것
    => 데이터에 대한 탐색에 가장 중요한

* ML에서 가장 큰 중요한 부분!
  -> EDA를 잘못하면 뻔한 내용밖에 안 나오고
     모델의 성능도 크게 향상이 없음!
  단, 정답은 없음! +데이터 + 관련 도메인지식  대회같은 경우 다양한 아이디어가 있음

In [2]:
import os
os.getcwd()

'C:\\Users\\NT551XCJ\\Documents\\suyoungkim\\05.DataHandling'

In [24]:
import pandas as pd
import numpy as np

path ='C:\\Users\\NT551XCJ\\Documents\\suyoungkim\\05.DataHandling'
file = path +'\\11_sales-funnel.xlsx'
data = pd.read_excel(file, sheet_name="Sheet1")
data.head(5)

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


* 목적 : 매출 관련 데이터를 중심으로 인사 평가를 해보자!  
데이터에 대한 간략 설명  
Account : 고객 게좌번호  
Name : 고객의 이름  
Rep : 영업 담당자(실무 영업사원)  
Manager : 영업 담당자의 팀장  
Product : 판매한 물건  
Quantity : 판매 수량  
Price : 판매 가격  
Status: 현재 진해 상황

## pivot

In [ ]:
# pivot_table 을 활용해서 새롭게 판을 짜는 형식!
# 1. 가로에 무엇을 볼지 
# 2. 세로에 무엇을 볼지  
# 3. 만들어진 공간을 어떻게 채울까에 대한 고민
#     -> 여러개의 데이터들을 어떻게 집계처리해서 볼지 고민할 것

# 참고) pandas에서 2.x과 1.x 버전이 좀 다름 2.x이 좀 더 엄밀하게 사용하도록 추구
# -> 예전에는 가로만 세팅하면 알아서 세로를 세팅하고, 값들을 알아서 수치형 컬럼대상 평균으로 계산해줬음
# -> 이제는 가로, 세로 세팅을 정석으로 해줄 필요가 있음

* pivot_table 에서  
index : 가로 세팅(기준)  
values : 세로 세팅(보고자하는 속성)  
aggfunc : 집계 기준(평균, etc)


In [8]:
# 팀장들에 대해서 팀에 속한 매출액을 보자
# -> 평가의 기준이 오로지 매출
# -> 값을 처리할지 얘기하지 않으면 : 평균

pd.pivot_table(data, index = ['Manager'], values = ['Price'])

,Price
Manager,
Debra Henley,26111.111111
Fred Anderson,35875.000000


In [10]:
# 팀당 총 매출액 확인해보자

pd.pivot_table(data, index = ["Manager"], values =["Price"], aggfunc = [np.sum])

# aggfunc 편히 사용하기 위해서는 np계열의 집계함수들을 활용. 
#   + 단 함수명만 제시해야함. ()호출은 pivot_table이 함


C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\403157068.py:3: FutureWarning: The provided callable <function sum at 0x0000024322DC49A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(data, index = ["Manager"], values =["Price"], aggfunc = [np.sum])


,sum
,Price
Manager,
Debra Henley,235000
Fred Anderson,287000


In [11]:
# 영업팀별로 사원에 대한 매출 총계는?
pd.pivot_table(data,index = ["Manager","Rep"], values =["Price"], aggfunc = [np.sum])

C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\3885130317.py:2: FutureWarning: The provided callable <function sum at 0x0000024322DC49A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(data,index = ["Manager","Rep"], values =["Price"], aggfunc = [np.sum])


sum
                              Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [13]:
# 꾸준히 실적을 낸 것인지
pd.pivot_table(data,index = ["Manager","Rep"], values =["Price"], aggfunc = [np.sum, len])

C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\1372722359.py:2: FutureWarning: The provided callable <function sum at 0x0000024322DC49A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(data,index = ["Manager","Rep"], values =["Price"], aggfunc = [np.sum, len])


sum   len
                              Price Price
Manager       Rep                        
Debra Henley  Craig Booker    80000     4
              Daniel Hilton  115000     3
              John Smith      40000     2
Fred Anderson Cedric Moss    110000     4
              Wendy Yule     177000     4

* 세로줄을 항목별로 나눠서 보고 싶을 때  
columns

In [17]:
# 구체적인 회사 상품별 판매 실적(영업사원별)
pd.pivot_table(data, index = ["Manager","Rep"], 
               values =["Price"], 
               columns = ["Product"],
               aggfunc = [np.sum])

C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\3481243934.py:2: FutureWarning: The provided callable <function sum at 0x0000024322DC49A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(data, index = ["Manager","Rep"],


sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

In [16]:
# 참고) 너무 많은 기준을로 표를 가로, 세로, 세팅을 하면  
#     -> 많은 누락된 셀이 발생할 수 있음  
#     -> df에서 해당하는 조건에 맞지 않은 데이터가 있어서 NaN 빵구난 데이터가 발생
#     -> 누락된 데이터 채우기 : fill_na, , dropna etc
# pivot_table 에서는 fill_value 파라미터로 세팅

# 구체적인 회사 상품별 판매 실적(영업사원별)
pd.pivot_table(data, index = ["Manager","Rep"], 
               values =["Price"], 
               columns = ["Product"],
               aggfunc = [np.sum],
                fill_value = 0)

C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\2524476111.py:8: FutureWarning: The provided callable <function sum at 0x0000024322DC49A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(data, index = ["Manager","Rep"],


sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

In [18]:
# 판매한 수량까지도 같이 보고 싶을 때
pd.pivot_table(data, index = ["Manager","Rep"], 
               values =["Price","Quantity"], 
               columns = ["Product"],
               aggfunc = [np.sum],
                fill_value = 0)

C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\3859123516.py:2: FutureWarning: The provided callable <function sum at 0x0000024322DC49A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(data, index = ["Manager","Rep"],


sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

In [19]:
# + 맨 밑에 총계
pd.pivot_table( data, index=["Manager","Rep"],
               values =["Price", "Quantity"], # --> 일반적으로 수치형컬럼
                columns=["Product"], # --> 카테고리형 컬럼
                aggfunc=[np.sum],
                fill_value=0,
                margins=True)

C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\2375023145.py:2: FutureWarning: The provided callable <function sum at 0x0000024322DC49A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table( data, index=["Manager","Rep"],
C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\2375023145.py:2: FutureWarning: The provided callable <function sum at 0x0000024322DC49A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table( data, index=["Manager","Rep"],
C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\2375023145.py:2: FutureWarning: The provided callable <function sum at 0x0000024322DC49A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep

sum                                       \
                              Price                                        
Product                         CPU Maintenance Monitor Software     All   
Manager       Rep                                                          
Debra Henley  Craig Booker    65000        5000       0    10000   80000   
              Daniel Hilton  105000           0       0    10000  115000   
              John Smith      35000        5000       0        0   40000   
Fred Anderson Cedric Moss     95000        5000       0    10000  110000   
              Wendy Yule     165000        7000    5000        0  177000   
All                          465000       22000    5000    30000  522000   

                                                                       
                            Quantity                                   
Product                          CPU Maintenance Monitor Software All  
Manager       Rep                                                      
Debra Henley  Craig Booker         2           2       0        1   5  
              Daniel Hilton        4           0       0        1   5  
              John Smith           1           2       0        0   3  
Fred Anderson Cedric Moss          3           1       0        1   5  
              Wendy Yule           7           3       2        0  12  
All                               17           8       2        3  30

In [21]:
# 집계처리시 price 평균, 총계, quantity 총계만 보고 싶음
# -> 쌍에 대한 구조! dict 사용


pd.pivot_table( data, index=["Manager","Rep"],
               values =["Price", "Quantity"], # --> 일반적으로 수치형컬럼
                columns=["Product"], # --> 카테고리형 컬럼
                aggfunc={
                    "Price" : [np.mean, np.sum],
                    "Quantity" : [np.sum]
                },
                fill_value=0,
                margins=True)

C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\3291020914.py:5: FutureWarning: The provided callable <function mean at 0x0000024322DC5A80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pd.pivot_table( data, index=["Manager","Rep"],
C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_16360\3291020914.py:5: FutureWarning: The provided callable <function sum at 0x0000024322DC49A0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table( data, index=["Manager","Rep"],


ValueError: The name None occurs multiple times, use a level number

## group by

sql에서 했던 group by 랑 똑같음  
기존의 데이터들을 어떻게 묶어서 볼 것인가의 새로운 기준  
& 여러 개의 값을 묶어서 표현 : 집계처리(대표값)  
sql) select avg(zumsu) from data group by class ~~  
pandas) df.group_by(by= ~).agg(집계처리)  
          df.group_by(by= ~).집계함수

In [26]:
file2 = path+'\\10_phone_data.csv'
data = pd.read_csv(file2, sep=',')
data.head(5)

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         830 non-null    int64  
 1   date          830 non-null    object 
 2   duration      830 non-null    float64
 3   item          830 non-null    object 
 4   month         830 non-null    object 
 5   network       830 non-null    object 
 6   network_type  830 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 45.5+ KB


In [ ]:
# 자주 사용되는 것 : 어떤 값이 몇 개가 있느냐?
# ---> df[~~~].value_counts()

In [28]:
data.loc[:, "item"].value_counts()
# --> item의 컬럼에 유니크한 값들의 종류과 갯수!!!!
#     + 정렬까지 내림차순으로 자동으로 보여 줌!

item
call    388
sms     292
data    150
Name: count, dtype: int64

In [29]:
# Q) duration  컬럼의 최대 사용한 시간은 얼마인가?

data.loc[:,"duration"].max()

10528.0

* 주로 체크할 때 : df.describe()

In [32]:
data.describe()

,index,duration
count,830.000000,830.000000
mean,414.500000,117.804036
std,239.744656,444.129560
min,0.000000,1.000000
25%,207.250000,1.000000
50%,414.500000,24.500000
75%,621.750000,55.000000
max,829.000000,10528.000000


In [34]:
# Q) item 항목들 중에서 전화인 call인 경우, 사용한 시간의 총합?
data[data["item"]=="call"]["duration"].sum()

92321.0

In [37]:
# 관찰 대상이 사용 건별이 아니라 월별로 보자
data.groupby(by="month").groups

{'2014-11': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...], '2014-12': [228, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, ...], '2015-01': [381, 386, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 3

In [38]:
data.groupby(by="month").groups.keys()

dict_keys(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'])

In [39]:
data["month"].unique()

array(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'],
      dtype=object)

In [ ]:
# df 에서 groupby 만 하면 안나타남. 구체적인 지시를 해줘야함

In [40]:
# Q) month별로 사용시간에 평균값을 구해본다면??
pd.pivot_table(data, index=["month"], values = ["duration"],
               aggfunc="mean")

,duration
month,
2014-11,115.823657
2014-12,93.260318
2015-01,88.894141
2015-02,113.301453
2015-03,225.251891


In [41]:
# 중요!
# df.groupby(~) : 누느에 보이지는 않아도 df처럼 생각하고 뒤에 취할 행동을 기술할 것!

In [43]:
# Q) item 컬럼의 값 중에서 종류가 call인 경우에 대해서
# network통신사별로 duration의 총 합을 구해보자

data[data["item"]=="call"].groupby(by="network")["duration"].sum()

network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

In [44]:
data[data["item"]=="call"].groupby(by="network").sum("duration")

,index,duration
network,,
Meteor,20894,7200.0
Tesco,29264,13828.0
Three,49405,36464.0
Vodafone,25664,14621.0
landline,22583,18433.0
voicemail,11350,1775.0


In [53]:
# 묶는 기준이 꼭 1개가 아니라 여러개일 수 있음
# month별로 item 에 대해서 데이터 숫자가 몇 건?

data.groupby(by =["month","item"])["date"].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

In [56]:
# 보려고 하는 집계함수에 대한 처리를 각각 하고자 할 때
# .agg({})
# 참고) pivot_table에서 aggfunc = {어느 속성: 무엇을 계산}
data.groupby(by=["month","item"]).agg(
    {
        "date" : "first",
        "duration":"count"
    }
)

date  duration
month   item                          
2014-11 call  15/10/14 06:58       107
        data  15/10/14 06:58        29
        sms   16/10/14 22:18        94
2014-12 call  14/11/14 17:24        79
        data  13/11/14 06:58        30
        sms   14/11/14 17:28        48
2015-01 call  15/12/14 20:03        88
        data  13/12/14 06:58        31
        sms   15/12/14 19:56        86
2015-02 call  15/01/15 10:36        67
        data  13/01/15 06:58        31
        sms   15/01/15 12:23        39
2015-03 call  12/02/15 20:15        47
        data  13/02/15 06:58        29
        sms   19/02/15 18:46        25

<!-- 원하는 기준대로 변형 : pivot_table, groupby

opt) 단순히 모양 자체를 변경! stack(), unstack()
-> 가로, 세로에 대한 변경을 할 때

이 후 파악한 관계들에 대한 효과적이 전달을 위한 시각화
-> 간단하게 파이썬에서 확인을 위해서 : matplotlib => seaborn, plotyly
    + pandas에서도 plot 이라는 메서드(matplotlib를 기반)가 있음
-> 전체적으로 다 보고서/DashBoard를 하려면, 전문적인 BI툴을 사용하는 것이 좋음, 태블로
-> 경영진/의사결정라인 의사결저엥 효율적인 전달!